# SQL basics (2)

In [16]:
import sqlite3
import pandas as pd

In [17]:
def exe(cursor: object, query: 'string'):
    cursor.execute(query)
    for row in  cursor.fetchall():
        print(row)

In [2]:
conn = sqlite3.connect('data/european-soccer.sqlite')
c = conn.cursor()

In [4]:
query = '''
SELECT *
FROM Country;
'''

exe(c, query)

(1, 'Belgium')
(1729, 'England')
(4769, 'France')
(7809, 'Germany')
(10257, 'Italy')
(13274, 'Netherlands')
(15722, 'Poland')
(17642, 'Portugal')
(19694, 'Scotland')
(21518, 'Spain')
(24558, 'Switzerland')


##### Explore the database

Having the possibility of asking the list of the tables and their columns in a database is useful. Halas there is no standardised instructions to do that in SQL, access the metadata of the database is rather an action executed at the level of the DBMS. Each DBMS has a different method. For exemple the commands – after logging into user account – `\dt` or `\dt+` in PostgreSQL, `show tables;` in MySQL, `.tables` in SQLite, `SELECT table_name FROM dba_tables;` in Oracle, `db2 list tables for schema schema_name` in IBM db2 or `SELECT * FROM information_schema.tables;` in Microsoft SQL Server. 

When using SQLite with Python there are other method to get those metadata, some or them are powerful, but you should expect to learn specific instructions that fit the environment and DBMS you will be working with in your project, company or client. Keep in mind that if even if SQLite is widely used – and that’s why you should know it –, it is not designed for a scalable production environment.

### `PRAGMA` statements

SQLite uses [PRAGMAs](https://www.sqlite.org/pragma.html) to define or changes some behavior of the engine. For example, the instruction `PRAGMA case_sensitive_like = TRUE` makes SQLite engine case sensitive (when using `LIKE` for example). You can also use `PRAGMA` to access to metadatas.

#### Get the list of tables

In [5]:
query = '''
PRAGMA table_list;
'''

exe(c, query)

('main', 'Team_Attributes', 'table', 25, 0, 0)
('main', 'Team', 'table', 5, 0, 0)
('main', 'Country', 'table', 2, 0, 0)
('main', 'League', 'table', 3, 0, 0)
('main', 'Match', 'table', 115, 0, 0)
('main', 'Player', 'table', 7, 0, 0)
('main', 'Player_Attributes', 'table', 42, 0, 0)
('main', 'sqlite_sequence', 'table', 2, 0, 0)
('main', 'sqlite_schema', 'table', 5, 0, 0)
('temp', 'sqlite_temp_schema', 'table', 5, 0, 0)


1. The first column is the [schema](https://database.guide/what-is-a-database-schema/) to which the table belong
2. The 2d column is the tables names
3. The 3d column is the table type. Most common are `table` and `view`. A view is a query written that is saved as… a view
4. The 4th column is the number of fields in tables
5. The 5th and 6th columns are options defined at the creation of the tables, no need to go that deep for the moment in this introductory course.

#### Get columns metadatas

The `PRAGMA table_info(<table_name>)` is used to get infos about columns metadatas of the table passed as argument :

In [6]:
query = '''
PRAGMA table_info(Country)
'''

exe(c, query)

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'name', 'TEXT', 0, None, 0)


1. The first column is the columns indexes
2. The 2d is the columns names
3. The 3d is the columns types
4. The 4th indicates if the columns can be NULL (0 = False = no, 1 = True = yes)
5. The 5th gives the columns default values (when defined)
6. The 6th indicates if the columns are primary keys

### Access to metadatas with Python

The Cursor() class has a `.description` attribute that corresponds to `table_info()` and thus contains the columns name :

In [7]:
c.description

(('cid', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None),
 ('notnull', None, None, None, None, None, None),
 ('dflt_value', None, None, None, None, None, None),
 ('pk', None, None, None, None, None, None))

It contains nothing until you place the cursor on a table by sending it a query (we will add a `LIMIT 0` to the query : we don’t really want an output, just place the cursor on the table we want to inspect).

In [8]:
c.execute('SELECT * FROM Country LIMIT 0')
c.description

(('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None))

We can extract column names with a comprehensive list to get a one-liner (we could have done the same with the `PRAGMA` method, of course) :

In [9]:
columns_names = [description[0] for description in c.description]
columns_names

['id', 'name']

The ability to get the columns list from within our script (whether via `PRAGMA` or Cursor() class) give us the possibility to adapt dynamically our queries to the current state of the database. There is numerous usecases when an app has to change the schema of a database it uses. In this case we can easily adapt automatically our queries to the modified schema. For exemple we don’t even know all the fields in the `Matches` table, we can automatically generate a query calling those columns :

In [10]:
c.execute('SELECT * FROM Match LIMIT 0')
columns_names = [description[0] for description in c.description]
query = 'SELECT ' + ', '.join(columns_names) + ' FROM Match'
print(query)

SELECT id, country_id, league_id, season, stage, date, match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, home_player_X1, home_player_X2, home_player_X3, home_player_X4, home_player_X5, home_player_X6, home_player_X7, home_player_X8, home_player_X9, home_player_X10, home_player_X11, away_player_X1, away_player_X2, away_player_X3, away_player_X4, away_player_X5, away_player_X6, away_player_X7, away_player_X8, away_player_X9, away_player_X10, away_player_X11, home_player_Y1, home_player_Y2, home_player_Y3, home_player_Y4, home_player_Y5, home_player_Y6, home_player_Y7, home_player_Y8, home_player_Y9, home_player_Y10, home_player_Y11, away_player_Y1, away_player_Y2, away_player_Y3, away_player_Y4, away_player_Y5, away_player_Y6, away_player_Y7, away_player_Y8, away_player_Y9, away_player_Y10, away_player_Y11, home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, 

Of course selecting all columns that way is useless, but we could write a more elaborated script that produces a query which select only the columns beginning by `'home_'` for example. Try to do this as an exercise :

In [11]:

c.execute('SELECT * FROM Match LIMIT 0')
columns_names = [description[0] for description in c.description]

# your code here :
# 1. create an empty list named columns_home
# 2. examine each element of the columns_names list and test if it begins with 'home_'
# 3. if the test is ok, append the element to colmuns_home

query = 'SELECT ' + ', '.join(columns_home) + ' FROM Match'
print(query)



Another exercice : create a function to get tables in a data base, and another to get columns in a table

In [ ]:
def get_table(cursor: object):
    # your code here
    # you can either use the attribute description of the cursor, or use the `PRAGMA` directive and call the function exe()
    
    table_list =
    return table_list

In [ ]:
def get_columns(cursor: object, table_name: 'string'):
    # your code here
    # # you can either use the attribute .description of the cursor – you’ll have to write a query this time, or use the `PRAGMA` directive and call the function exe()

    columns_list =
    return columns_list



Test your function, and then close the connector and cursor.

In [13]:
c.close()
conn.close()

## Create a database

### From scratch

Let’s create a database about movies. The database will be made of two tables : the table movies, which contains information like Title, budget, vote obtained… and the table credits, which contains information about the cast, the producer…

Here the ERD :

![ERD of movies database](./images/Movies-ERD.png)

The steps to create a database from scratch are :

1. Create the base along with a connector
2. Create a cursor
3. Execute `CREATE TABLE` instructions, specifying fields/column names, their type, and if needed if a column is a `PRIMARY KEY` and other options
4. In comparison of queries, you don’t just execute a `CREATE TABLE` instruction, you hav to "commit" it, transmit it to the database via the connector with the `.commit()` methods
5. Once the table is created, it’s just a container : content (data) must be gathered and inserted into the database wit the `INSERT INTO` instructions indicating the `VALUES` to be inserted

Note : SQLite has very limited dataypes (5 classes) : `NULL`, `INTEGER`, `REAL`(float), `TEXT` and `BLOB`, while you have around 45 types in PostgreSQL for example, several types of int, float, char, byte types, and more structured types like timestamp, geometry and even XML, JSON, IP address… Such precision about the type (for example int of different sizes : from two bytes to height) improves security and efficiency : it saves a lot of space to use int on 2 bytes rather 8 if it represents small enough values…

Blobs is for data stored exactly as it was input. Dates can be stored as int, real or text, according to different methods and functions available, respectivelly as Unix Time (number of seconds since first of january 1970), as Julian day number or ISO8601 strings (`'YYYY-MM-DD HH:MM:SS,SSS'`)

A last thing a bit strange for a database is that the columns are weakly typed : you can insert a `TEXT` value in an `INT` column and won’t get issued an error. On the other hand, it allows to optimize the space taken by the values in terms of storage.

In [14]:
# connector creation
conn_mdb = sqlite3.connect('data/movies.db')
# cursor creation
c_mdb = conn_mdb.cursor()

Instructions to create the `Movies` table :

In [15]:
creation_instructions = '''
CREATE TABLE IF NOT EXISTS Movies(
    Id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
    Title TEXT,
    Date TEXT,
    Duration INTEGER,
    Budget INTEGER,
    First_week_viewers INTEGER,
    Votes REAl
    );
'''

Some keywords explanations :

* `IF NOT EXISTS` : the table is created only if it doesn’t exist yet
* `PRIMARY KEY` : identifies the field as a primary key (thank you captain obvious !)
* `AUTOINCREMENT` : if a new line is created or added, the primary key (PK) value will be automatically incremented by one
* `UNIQUE` : puts a constraint ont this field, each Id value must be unique (by definition of a PK) so if you add a line where the Id is a value already used, it will return an error. A lot of keyword at this stage are just constrains to avoid mistake or incoherence.
*  `NOT NULL` : another example of constraint

Once the isntructions are written, execute those instructions and commit them to the database :


In [16]:
c_mdb.execute(creation_instructions)
conn_mdb.commit()

Using the function you created above, verify that we now have a database and a `Movies` table :

In [1]:
# your code here


Now verify the columns of the `Movies` table with another function you wrote :

In [2]:
# your code here


Let’s put some data in it ! Remember : a record (a line) in a table is a *tuple*, data will be a list of tuple :

In [19]:
data = [(1, 'A good movie', '2024-12-04', 120, 2000000, 259023, 4.36),
        (2, 'Another good movie, slightly better', '2024-12-05', 110, 500000, 354352, 4.63),
        (3, 'A bad movie, but with some success', '1985-01-01', 84, 600000, 165904, 4.26),
        (4, 'A very bad movie', '2005-04-25', 93, 1000000, 235, 2.86),
        (5, 'A not so bad movie', '2019-03-23', 104, 1500000, 40334, 3.86)]

Data will be inserted line by line (using a for loop) with the command `INSERT INTO`:

In [20]:
for d in data:
    c_mdb.execute('''
    INSERT INTO Movies(id, Title, Date, Duration, Budget, First_week_viewers, Votes)
    VALUES(?, ?, ?, ?, ?, ?, ?);''', d)

Don’t forget to commit:

In [21]:
conn_mdb.commit()

Of course we don’t forget to close connector and cursor :

In [22]:
c_mdb.close()
conn_mdb.close()

Let’s verify nothing went wrong by printing the content of the table : 

In [23]:
# re-open the database
conn_mdb = sqlite3.connect('data/movies.db')
c_mdb = conn_mdb.cursor()

#query all the lines
query = 'SELECT * FROM Movies'
exe(c_mdb, query)

(1, 'A good movie', '2024-12-04', 120, 2000000, 259023, 4.36)
(2, 'Another good movie, slightly better', '2024-12-05', 110, 500000, 354352, 4.63)
(3, 'A bad movie, but with some success', '1985-01-01', 84, 600000, 165904, 4.26)
(4, 'A very bad movie', '2005-04-25', 93, 1000000, 235, 2.86)
(5, 'A not so bad movie', '2019-03-23', 104, 1500000, 40334, 3.86)


### From a .csv file

If it can be in some situation convenient to add few data to a table by hardcoding them in a script, it becomes quickly boring and confusing. In most cases, data are availables in flatfiles like `.csv` that favorises accessibility by its simplicity. Pull the data from a `.csv` in a database secures and improves the efficiency of data processing and diffusion.

The `Credits` table contains 8 columns :
```
Id;Movie_id;Direction;Producer;Studio;Playscreen;Cast;Country
1;3;"Big director";"Big producer";"Big studio";"Big screenwriter";"Big Actor 1, Big Actor 2, Other big actors";"Big country"
2;1;"Unknown director";"Unknown producteur";"Unknown studio";"Unknown screenwriter";"Unknown actor 1, Unknown acteur 2, Other unknown actors";"Unknown country"
3;2;"Small director";"Small producer";"Small studio";"Small screenwriter";"Small actor 1, Small actor 2, Small other actors";"Small country"
4;5;"Acceptable director";"Acceptable producer";"Acceptable studio";"Acceptable screenwriter";"Acceptable actor 1, Acceptable actor 2, Other acceptable actors";"Acceptable country"
5;4;"Incompetent director";"Incompetent producer";"Incompetent studio"; "Incompetent screenwriter";"Incompetente actor 1, Incompetent actor 2, Other incompetent actors";"Incompetent country"
```

Let’s see how we can get the data for our second table, `Credits`, from this `.csv` file. The difficult part is that we have to parse data (identify which part of the files corresponds to what information and where it should go in the database). Each line must be decomposed in values matching the fields and their types. Everything in the file will be loaded as a string type ! We’ll have to split and cast. 

1. Declare data as an empty list : it will be a list of lists, each column will be stored as list
2. Open the `data/credits.csv` file, get rid of the headers, reading it line by line, split each line (separator is `';'` in our example) and
3. Create the Movies table. Declare all the column as before, indicating their type, even the column that will be the foreign key (`Movies_id`). We’ll just add the constraint `NOT NULL` to this declaration, to be sure that we dont have credits records in this table and no related movie in the table `Movies`.
4. To declare the `Movies_id` column as a foreign key (FK), we write another line with the keywords `FOREIGN KEY`, specifying between parentheses which column is the FK, then indicate with the keyword `REFERENCES` to which table and PK it relates :

In [24]:
data = [] 
with open('data/credits.csv', 'r') as f:
    f.readline() # get rid of the first line containing headers - we could test if headers match columns name
    for line in f:
        data.append(line.rstrip().split(';')) 

for d in data:
    d[0] = int(d[0]) # cast the id (primary key) to int 
    d[1] = int(d[1]) # cast the movie_id (foreign key) to int 

# creation instructions 
create_moviestable = '''
CREATE TABLE IF NOT EXISTS Credits(
    Id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
    Movie_id INTEGER NOT NULL,
    Direction TEXT,
    Producer TEXT,
    Studio TEXT,
    Playscreen TEXT,
    Cast TEXT,
    Country TEXT,
    FOREIGN KEY (Movie_id)
        REFERENCES Movies (Id)
);
''' 

# table creation
c_mdb.execute(create_moviestable)
conn_mdb.commit()

# pull data in database

# instructions
insert_instructions = '''
INSERT INTO Credits(Id,
                    Movie_id,
                    Direction,
                    Producer,
                    Studio,
                    Playscreen,
                    Cast,
                    Country)
    VALUES(?, ?, ?, ?, ?, ?, ?, ?);
'''
for d in data:
    c_mdb.execute(insert_instructions, d)
conn_mdb.commit()

In [27]:
query = 'SELECT * FROM Credits'
exe(c_mdb, query)

(1, 3, '"Big director"', '"Big producer"', '"Big studio"', '"Big screenwriter"', '"Big Actor 1, Big Actor 2, Other big actors"', '"Big country"')
(2, 1, '"Unknown director"', '"Unknown producteur"', '"Unknown studio"', '"Unknown screenwriter"', '"Unknown actor 1, Unknown acteur 2, Other unknown actors"', '"Unknown country"')
(3, 2, '"Small director"', '"Small producer"', '"Small studio"', '"Small screenwriter"', '"Small actor 1, Small actor 2, Small other actors"', '"Small country"')
(4, 5, '"Acceptable director"', '"Acceptable producer"', '"Acceptable studio"', '"Acceptable screenwriter"', '"Acceptable actor 1, Acceptable actor 2, Other acceptable actors"', '"Acceptable country"')
(5, 4, '"Incompetent director"', '"Incompetent producer"', '"Incompetent studio"', ' "Incompetent screenwriter"', '"Incompetente actor 1, Incompetent actor 2, Other incompetent actors"', '"Incompetent country"')


Of course we close connector and cursor :

In [28]:
c_mdb.close()
conn_mdb.close()

### From a dataframe

We will process most of the data using dataframes. Moreover, `pandas` allows to load data from multiple sources − saving us from writing boilerplate to load different formats and address a lot of exceptions, difficulties, etc. Therefore it is important to know how to push data to a database directly from a dataframe.
`df.to_sql(<table name>, <connector name>)` is a method to , providing the table name and connector to the database. Providing wo other optionnal arguments may be useful : `if_exists='replace'` to replace the table if a table with the same name already exists (`'fail'` by default, you can also chose `'append'`), and `index=False` if the dataframe index has no interest (`True` by defaults, a column `index_label` will be created).

1. load data in a dataframe (if it does not already exists)
2. create a connector to a database (will be created if doesn’t exist yet)
3. use the method `.to_sql()`. Returns the number of rows written.

Let’s forget mock-up data for this part, and download real data on [Kaggle](https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata?resource=download). Take the time to read context and metadata.

In [5]:
df = pd.read_csv("data/TMDB-moviedatabase/tmdb_5000_movies.csv")

conn_mdb = sqlite3.connect('data/tmdb_movies.db')
df.to_sql('Movies', conn_mdb, if_exists='replace', index=False)

4803

In [10]:
c_mdb = conn_mdb.cursor()
show = 'SELECT * FROM Movies LIMIT 1' # always put a limit 
exe(c_mdb, show)

(237000000, '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]', 'http://www.avatarmovie.com/', 19995, '[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]', 'en', 'Avatar', 'In the 22nd century, a paraplegic Marine is dispatched t

In [11]:
columns = 'PRAGMA table_info(Movies)'
exe(c_mdb, columns)

(0, 'budget', 'INTEGER', 0, None, 0)
(1, 'genres', 'TEXT', 0, None, 0)
(2, 'homepage', 'TEXT', 0, None, 0)
(3, 'id', 'INTEGER', 0, None, 0)
(4, 'keywords', 'TEXT', 0, None, 0)
(5, 'original_language', 'TEXT', 0, None, 0)
(6, 'original_title', 'TEXT', 0, None, 0)
(7, 'overview', 'TEXT', 0, None, 0)
(8, 'popularity', 'REAL', 0, None, 0)
(9, 'production_companies', 'TEXT', 0, None, 0)
(10, 'production_countries', 'TEXT', 0, None, 0)
(11, 'release_date', 'TEXT', 0, None, 0)
(12, 'revenue', 'INTEGER', 0, None, 0)
(13, 'runtime', 'REAL', 0, None, 0)
(14, 'spoken_languages', 'TEXT', 0, None, 0)
(15, 'status', 'TEXT', 0, None, 0)
(16, 'tagline', 'TEXT', 0, None, 0)
(17, 'title', 'TEXT', 0, None, 0)
(18, 'vote_average', 'REAL', 0, None, 0)
(19, 'vote_count', 'INTEGER', 0, None, 0)


In [12]:
c_mdb.close()
conn_mdb.close()

Of course, the reverse is also possible, you can read data from a database and store it into a dataframe, with the `pd.read_sql()` method, providing it a query and a connector. A useful optional argument is `index_col`, column(s) to set as dataframe index :

In [18]:
conn_sdb = sqlite3.connect('data/european-soccer.sqlite')
c_sdb = conn_sdb.cursor()

selection = '''
SELECT *
FROM Country;
'''

df = pd.read_sql(selection, conn_sdb, index_col='id')
df

,name
id,
1,Belgium
9,Malta
1729,England
4769,France
7809,Germany
10257,Italy
13274,Netherlands
15722,Poland
17642,Portugal


With a more complex query : 

In [22]:
selection = '''
SELECT p.player_name, pa.overall_rating
FROM Player_Attributes AS pa
JOIN Player AS p ON pa.id = p.id
ORDER BY overall_rating DESC
LIMIT 20;
'''

df = pd.read_sql(selection, conn_sdb)
df

,player_name,overall_rating
0,Manu Molina,91
1,Manu Torres,91
2,Fede Vico,89
3,Manu Lanzarote,88
4,Lorenzo Pique,87
5,Lorenzo Squizzi,87
6,Lorenzo Stovini,87
7,Lorenzo Tonelli,87
8,Miguel Portillo,87
9,Faysel Kasmi,86


If you expect to need or plan to use Pandas and relationnal databases, you should get familiar with [SQLalchemy](https://www.sqlalchemy.org/), an [Object Relational Mapper](https://en.wikipedia.org/wiki/Object%E2%80%93relational_mapping). That’s also true for any context implying a database with Python, actually.

## Other SQL keywords

Here some keywords that will be used to perform useful if no vital operations : change values in the database, suppress tables, combine several queries…

### `UPDATE`

One major function of a database is to keep data updated (the 'U' in 'CRUD'). Sooner or later you will need to update values, with – O surprise – the `UPDATE` statement `[ ]` are optional instructions :

```SQL
UPDATE <table>
SET
  <column1_to_update> = <value1>,
  <column2_to_update = <value2>,
    …
[WHERE
  <conditions_to_select_cells>]
ORDER BY <column_to_consider>
LIMIT <number_of_lines> ;
```

`ORDER BY` and `LIMIT` are mandatory : we don’t know in which order data is sotred in tables, so we have to ask for a "reference order", and limit the number of operation. To process all the table, just set the `LIMIT` to `-1`

We will see an example below.

If you want to update data from values taken from another table, use the `UPDATE FROM` statement : 

```SQL
UPDATE <table>
SET
  <column1_to_update> = <value1>,
  <column2_to_update = <value2>,
FROM
  <query> or <source_table>
[WHERE
  <conditions_to_select_cells>]
```


### `DELETE` and `DROP`

Delete data (the 'D' in 'CRUD') is very simple, it works like a `SELECT`, except that you delete the selected data :

```SQL
DELETE FROM <table_with_data_to_delete>
WHERE <search_condition>;
```

Notes : 
1. the entire records matching the condition will be deleted. If you want to delete a value in a record, what you really want to do is to `UPDATE` this particular value to `SET` it to `NULL`.
2. if you don’t specify a condition, all the records in the table will be deleted (the table will be emptied).
3. if you want to delete a table, you have to use the instruction `DROP` instead :\
   `DROP TABLE [IF EXISTS] [database_name.]table_name;`\
   (add `IF EXISTS` if you want to avoid errors in case the table doesn‘t exist.)
5. Deleting data may violate constrains defined on a table : an error will be issued. It can also be the case for a `DROP TABLE` because SQLite will implicitely delete data before the `DROP`. If this occurs, check and change the constrains before deleting or dropping.

### `UNION`

When you perform a query, a result set of data is returned. The `UNION` operator allows you to combine the result of two queries in one set :

In [21]:
union = '''
SELECT id, name
FROM Country

UNION

SELECT id, name
FROM League;
'''

exe(c_sdb, union)

(1, 'Belgium')
(1, 'Belgium Jupiler League')
(9, 'Malta')
(1729, 'England')
(1729, 'England Premier League')
(4769, 'France')
(4769, 'France Ligue 1')
(7809, 'Germany')
(7809, 'Germany 1. Bundesliga')
(10257, 'Italy')
(10257, 'Italy Serie A')
(13274, 'Netherlands')
(13274, 'Netherlands Eredivisie')
(15722, 'Poland')
(15722, 'Poland Ekstraklasa')
(17642, 'Portugal')
(17642, 'Portugal Liga ZON Sagres')
(19694, 'Scotland')
(19694, 'Scotland Premier League')
(21518, 'Spain')
(21518, 'Spain LIGA BBVA')
(24558, 'Switzerland')
(24558, 'Switzerland Super League')


**ATTENTION**, there are rules.
Try the following code :

In [ ]:
union = '''
SELECT *
FROM Country

UNION

SELECT *
FROM League;
'''

exe(c_sdb, union)

What did you get ? What did you expect ?

Union rules :
* the number of columns in the two sets of results must be the same
* columns datatypes must be compatibles (can’t mix `INT` and `TEXT` in one column for example)
* results columns name will be the columns name of the first query
* `ORDER BY` will be applied to the combined result set (test it)

There is an `UNION ALL` operator. Compare the results of `UNION` vs. `UNION ALL` on the request `SELECT Id FROM` with the `Country` then `League` tables. What’s the difference ?

In [ ]:
# Your code here






##### `DEFAULT` 

We have seen at the begining that default values can be specified when we create a column. The syntax is very simple :

```SQL
CREATE TABLE with_default (
  id INTEGER PRIMARY KEY,
  text1 TEXT,
  number REAL DEFAULT 3.14,
  text2 TEXT DEFAULT 'some_text'
);
```

When inserting data with this line :

```SQL
INSERT INTO with_default (text1) VALUES ('text');
```

Then when you request the record in the table `with_default`, you will have a tuple :\
`(1, text, 3.14, some_text)`

If you want to change the default values defined at the creation of the table, you have the instruction `ALTER TABLE` in SQL. In SQLite, it is more recommanded to create another table with the new default value, and copy the old table in the new one. Then drop the old table and rename the new table.

Exercise : create a table of your choice (for these exercises create a different database, like `exercises.sqlite` or `test.sqlite`, to keep other databases clean). In this table, define some columns with default values, insert different records (overwriting or not default values), show the records. Change the default value (follow the procedure described above), insert a new record to verify that the new default value works. Then delete records and finally drop the table. Example of data that could inspire you : employees, orders, students…

In [22]:
# your code here




### String functions

Aside of the numeric functions (`COUNT()`, `SUM()`, `AVG()`, `MIN()`, `MAX()`…), there is a set of functions that operates on strings, very useful for data preparation. These functions are generally used to define conditions, or with sub-requests (which are introduced at the end of this notebook).

For example, the `LENGTH()` function (no need to explain what it does ?) :

In [9]:
query = '''
SELECT LENGTH('notebook')
'''
exe(c_sdb, query)



(8,)


More elaborated query :

In [12]:
concat = '''
SELECT
    UPPER(p.player_name) || ' : ' || pa.overall_rating AS players_rating
FROM Player_Attributes AS pa
JOIN Player AS p ON pa.id = p.id
ORDER BY overall_rating DESC
LIMIT 20;
'''
exe(c_sdb, concat)

('MANU MOLINA : 91',)
('MANU TORRES : 91',)
('FEDE VICO : 89',)
('MANU LANZAROTE : 88',)
('LORENZO PIQUE : 87',)
('LORENZO SQUIZZI : 87',)
('LORENZO STOVINI : 87',)
('LORENZO TONELLI : 87',)
('MIGUEL PORTILLO : 87',)
('FAYSEL KASMI : 86',)
('FAZLI KOCABAS : 86',)
('LORENZO PELLEGRINI : 86',)
('MIGUEL PEDRO : 86',)
('PAWEL WSZOLEK : 86',)
('PAWEL ZIELINSKI : 86',)
('PECKS : 86',)
('PEDRAO : 86',)
('PEDRINHO : 86',)
('PEDRO ALVES : 86',)
('PEDRO MOREIRA : 86',)


Take the time to understand how work the combined string functions below, to implement the same behavior as a `RIGHT()` function (that do not exist in SQLite).

In [7]:
extract_name = '''
SELECT
    SUBSTR(p.player_name,
        INSTR(p.player_name, ' ') + 1,
        LENGTH(p.player_name))
FROM
    Player as p
LIMIT 20
'''
exe(c_sdb, extract_name)

('Appindangoye',)
('Cresswell',)
('Doran',)
('Galindo',)
('Hughes',)
('Hunt',)
('Kuhl',)
('Lennon',)
('Lennox',)
('Meijers',)
('Mokoena',)
('Mooy',)
('Muirhead',)
('Niguez',)
('Ramsey',)
('Splaine',)
('Taylor-Sinclair',)
('Wilbraham',)
('Chahechouhe',)
('Ba',)


Other classical string functions are available :

* `UPPER(<str>)`
* `LOWER(<str>)`
* `REPLACE(<text>, <text_to_be_replaced>, <replacing_text>)`
* `TRIM(<str>)` -> suppress leading and trailing empty chars. `LTRIM()` and `RTRIM()` also exist.
*  the concat operator between two strings is `||`
* `SUBSTR(<str>, <start_position>, <substring_length>)` -> returns a substring. Attention, position starts at 1 (not 0)!
* `INSTR(<str>, <sub-str>)` -> search for a substring and return its position
etc.

Some function are available in SQL but not implemented in SQLite, like `CONCAT()` or `CONCAT_WS()`, `LEFT()`, `RIGHT()`… ! Verify in the doc.

Exercice : test some string functions (for example, in a condition : extract firstname of player, and print only the players whose firstname is longer than N characters – choose N wisely) !


In [3]:
# your code here



### Datetime functions

SQLite has no datetime types, but has functions to manipulate date and time values. Temporal data are inherently complex to process : there is lot of way of representing it, different datatypes (string, int, objects…), different points of reference, calendars, and a datetime is made of different layers of values expressed in different bases (60 for seconds and minutes, 24 for hours, 365 for days…).

`DATETIME()` is a function that takes a datetime variable and applies (or not) modifiers to it (list [here](https://www.sqlite.org/lang_datefunc.html#modifiers)). Modifiers can be amounts of time added or removed, specific times – like first day of month, of year… –, or reference (to convert from one system to another). It , can help us to do a lot of things : 
* extract a datetime from a date and time string\
  `DATETIME('2025-01-17 09:15:32.156')`
* get the current datetime (UTC)\
  `DATETIME('now')` you should add the `'localtime'` modifier to get the… localtaime : `DATETIME('now', 'localtime')`\
  Note : `CURRENT_TIMESTAMP` is a keyword that returns current datetime in UTC.
* add or remove certain amount of time :\
  `DATETIME('now', '-1 months', 'localtime')`
* convert datetime values. For example, from string format to juliancalendar :
  `DATETIME(JULIANDAY(2025-01-17 09:15:32.156))`

We won’t detail all datetime functions here, see the manual. Here a list of some functions :

* `DATE()`
* `TIME()`
* `STRFTIME()` -> convert a datetime to a specified string format, most of them being the formats of the `strftime()` function in C (see [here](https://www.man7.org/linux/man-pages/man3/strftime.3.html)).
* `JULIANDAY()`
* `UNIXEPOCHTIME()`
* `CURRENTDATE()`
* `CURRENTTIME()`

As an exercise test some the above function :
1. Create an `order` table with few lines of mock-up data (for example two colums `object` and `price` + PK) and a order_date column with `CURRENT_TIME` as a default value.
2. Create an `invoice` table with a column `payment_due` that is the date of order + 30 days.

In [ ]:
# your code here






## The big part : `join`

Ressources : the [Wikipedia article on joins](https://en.wikipedia.org/wiki/Join_%28SQL%29) is very clear !

We do a join when we query several tables at once, tables beeing linked by relationships between keys, the primary key in one table being a foreign key in the other. A join operation selects records for which the primary key value in one table is equal to the value of a foreign key in another table, and output this matching data. That’s where the different kind of relationships (one-to-one, one-to-many, etc.) have an impact. 

In the first lecture, we outlined how much the underlying model of databases where formally inspired by set theory. Joins can be seen as operation on sets (of data). Each operation define a different way to cross-reference, or match, data from different tables. The different joins are :

* inner join
* left and right join
* outer join (or exclusive join)
* cross join
* self join

and we can represent most of them with Venn diagramms.

![Wenn diagramm of joins](./images/sql_joins.png)

Those are theoretical joins, not all of them are implemented in all DBMS. For example `LEFT JOIN` is rarelly implemented as it is equivalent to a `RIGHT JOIN` (with the tables taken in reverse order). Same for the different `OUTER JOIN` as they can be implemented just by adding a `WHERE` clause. For example SQLite do not implement `FULL OUTER JOIN`.

Let’s see the differences in details with SQLite.

Ressource : the SQLite doc pages about [syntax for joins](https://sqlite.org/lang_select.html#rjoin)

In [3]:
conn_sdb = sqlite3.connect('data/european-soccer.sqlite')
c_sdb = conn_sdb.cursor()

### Inner Join

The default join. Only returns the elements for which foreign and primary key match. Element in B or A without counterparts in the other table are ignored.

![Inner join illustration with Venn diagram](./images/inner_join.png)

Recall of the columns name :

In [13]:
query = '''
PRAGMA table_info(Country);
'''

exe(c_sdb, query)

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'name', 'TEXT', 0, None, 0)


In [9]:
query = 'SELECT * FROM Country;'
exe(c_sdb, query)

(1, 'Belgium')
(1729, 'England')
(4769, 'France')
(7809, 'Germany')
(10257, 'Italy')
(13274, 'Netherlands')
(15722, 'Poland')
(17642, 'Portugal')
(19694, 'Scotland')
(21518, 'Spain')
(24558, 'Switzerland')


Recall of the columns name :

In [15]:
query = '''
PRAGMA table_info(League);
'''

exe(c_sdb, query)

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'country_id', 'INTEGER', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)


In [11]:
query = 'SELECT * FROM League'
exe(c_sdb, query)

(1, 1, 'Belgium Jupiler League')
(1729, 1729, 'England Premier League')
(4769, 4769, 'France Ligue 1')
(7809, 7809, 'Germany 1. Bundesliga')
(10257, 10257, 'Italy Serie A')
(13274, 13274, 'Netherlands Eredivisie')
(15722, 15722, 'Poland Ekstraklasa')
(17642, 17642, 'Portugal Liga ZON Sagres')
(19694, 19694, 'Scotland Premier League')
(21518, 21518, 'Spain LIGA BBVA')
(24558, 24558, 'Switzerland Super League')


Execution of the inner join to show which League takes place in which country :

In [27]:
inner_join = '''
SELECT l.name, c.name
FROM League AS l
JOIN Country AS c ON l.country_id = c.id;
'''
exe(c_sdb, inner_join)

('Belgium Jupiler League', 'Belgium')
('England Premier League', 'England')
('France Ligue 1', 'France')
('Germany 1. Bundesliga', 'Germany')
('Italy Serie A', 'Italy')
('Netherlands Eredivisie', 'Netherlands')
('Poland Ekstraklasa', 'Poland')
('Portugal Liga ZON Sagres', 'Portugal')
('Scotland Premier League', 'Scotland')
('Spain LIGA BBVA', 'Spain')
('Switzerland Super League', 'Switzerland')


A country without associated League would not have been listed here (same for league, but it would be strange to have a league not associated with a country). Let’s build a table League_null with missing countries. That will be an excuse to see how to copy a table, and to change some values.
1. Create a new table
2. Copy the value by selecting them in the original table 

In [20]:
create = '''
CREATE TABLE IF NOT EXISTS League_null
(
Id INTEGER,
Country_id INTEGER,
Name TEXT
);
'''
exe(c_sdb, create)
conn_sdb.commit()

copy = '''
INSERT INTO League_null
(
Id,
Country_id,
Name
)

SELECT Id, Country_id, Name
FROM League;
'''
exe(c_sdb, copy)
conn_sdb.commit()

In [22]:
show = 'SELECT * FROM League_null'
exe(c_sdb, query)

(1, 1, 'Belgium Jupiler League')
(1729, 1729, 'England Premier League')
(4769, 4769, 'France Ligue 1')
(7809, 7809, 'Germany 1. Bundesliga')
(10257, 10257, 'Italy Serie A')
(13274, 13274, 'Netherlands Eredivisie')
(15722, 15722, 'Poland Ekstraklasa')
(17642, 17642, 'Portugal Liga ZON Sagres')
(19694, 19694, 'Scotland Premier League')
(21518, 21518, 'Spain LIGA BBVA')
(24558, 24558, 'Switzerland Super League')


3. modifying some values (we set 2 values in `Country_id` at `NULL` :

In [24]:
modify = '''
UPDATE League_null
SET Country_id = NULL
WHERE Id = 1729 OR Id = 4769
ORDER BY Id
LIMIT -1;
'''
exe(c_sdb, modify)
conn_sdb.commit()
exe(c_sdb, show)

(1, 1, 'Belgium Jupiler League')
(1729, None, 'England Premier League')
(4769, None, 'France Ligue 1')
(7809, 7809, 'Germany 1. Bundesliga')
(10257, 10257, 'Italy Serie A')
(13274, 13274, 'Netherlands Eredivisie')
(15722, 15722, 'Poland Ekstraklasa')
(17642, 17642, 'Portugal Liga ZON Sagres')
(19694, 19694, 'Scotland Premier League')
(21518, 21518, 'Spain LIGA BBVA')
(24558, 24558, 'Switzerland Super League')


* `UPDATE <table name>` : this keyword introduce a modification of a value in the named table
* `SET <column_1> = <new_value_1>, <column_2> = <new_value_2>` : we set the new values (we can update several columns)
* `WHERE <conditions>` : select the lines to update according to our conditions
* `GROUP BY <column>` : **mandatory** as we don’t know in which order are stored the lines in the database, we have to ask an order in which the lines will be processed
* `LIMIT <number of lines` : **mandatory** we have to set a limit of the number of lines that will be processed. If we want to process all the rows, set a negative value : every row that meet the conditions will be updated.

Exercise : make an inner join between the `Country` table and the `League_null` table. Note the difference : which lines disappeared ?

In [30]:
# Your code here


### Left/Right (outer) Join

As we said previously, it’s not unusual that left join only is implemented in DBMS, as right join is just the reverse. In SQLite `RIGHT JOIN` is implemented. This type of join covers elements that would have been selected in an inner join only, plus the remaining one belonging to the table on which the `SELECT` is performed :

![Left join (inclusive) illustration with Venn diagram](./images/left_join_inclusive.png) 

In [5]:
left_join = '''
SELECT 
    l.name, c.name
FROM 
    League_null as l
LEFT JOIN 
    Country as c ON c.id = l.country_id;
'''
exe(c_sdb, left_join)

('Belgium Jupiler League', 'Belgium')
('England Premier League', None)
('France Ligue 1', None)
('Germany 1. Bundesliga', 'Germany')
('Italy Serie A', 'Italy')
('Netherlands Eredivisie', 'Netherlands')
('Poland Ekstraklasa', 'Poland')
('Portugal Liga ZON Sagres', 'Portugal')
('Scotland Premier League', 'Scotland')
('Spain LIGA BBVA', 'Spain')
('Switzerland Super League', 'Switzerland')


Note how the query has been formatted with line breaks to isolate clauses. This is a good practice, which improves readability.

The query returns all the `League_null` elements, even if not all the corresponding countries are entered in the table. A `None` (or `NULL`) value is returned in those cases. We can define a default value to be returned instead with `COALESCE` : 

In [7]:
default_left_join = '''
SELECT 
    l.name,
    COALESCE(c.name, 'Unknown')
FROM 
    League_null as l
LEFT JOIN 
    Country as c ON c.id = l.country_id;
'''

exe(c_sdb, default_left_join)

('Belgium Jupiler League', 'Belgium')
('England Premier League', 'Unknown')
('France Ligue 1', 'Unknown')
('Germany 1. Bundesliga', 'Germany')
('Italy Serie A', 'Italy')
('Netherlands Eredivisie', 'Netherlands')
('Poland Ekstraklasa', 'Poland')
('Portugal Liga ZON Sagres', 'Portugal')
('Scotland Premier League', 'Scotland')
('Spain LIGA BBVA', 'Spain')
('Switzerland Super League', 'Switzerland')


As shown in the Venn diagram illustration below, an exclusive left join is obtained simply by adding a selection condition :

![Left join (exclusive) illustration with Venn diagram](./images/left_join_exclusive.png)

In [8]:
exclusive_left_join = '''
SELECT 
    l.name,
    COALESCE(c.name, 'Unknown')
FROM 
    League_null as l
LEFT JOIN 
    Country as c ON c.id = l.country_id
WHERE
    l.country_id IS NULL;
'''

exe(c_sdb, exclusive_left_join)

('England Premier League', 'Unknown')
('France Ligue 1', 'Unknown')


Exercise : write RIGHT JOIN clauses (inclusive and exclusive)

In [21]:
# your code here




### Full (outer) join

A full join (inclusive) combines rows of two tables relatively to a common column (usually PK in one table, and FK in the other) as seen in the joins before, but it return **all** the rows of the two tables. Rows that don’t match are also selected, with missing "counterpart" values at `NULL`. Harder to describe than to represent with a simple diagram :

![Full join (inclusive) illustration with Venn diagram](./images/full_join_inclusive.png)

By chance, `FULL (OUTER) JOIN` is implemented in SQLite. The `OUTER` keyword is optional. But before testing it, we should add somme records in the `Country` table, that are not referenced in the `League_null` table, to see what happen to records of `Country` that are not related to `League_null`, and vice versa.

In [25]:
# add at least one country to Country that is not referenced in the League table

# your code here
add = '''
INSERT INTO Country(id, name)
    VALUES(?, ?);
'''

c_sdb.execute(add, (9, 'Malta'))
conn_sdb.commit()

In [26]:
exe(c_sdb, 'SELECT * FROM Country')

(1, 'Belgium')
(9, 'Malta')
(1729, 'England')
(4769, 'France')
(7809, 'Germany')
(10257, 'Italy')
(13274, 'Netherlands')
(15722, 'Poland')
(17642, 'Portugal')
(19694, 'Scotland')
(21518, 'Spain')
(24558, 'Switzerland')


In [28]:
# Full join

full_join = '''
SELECT 
    l.name, 
    c.name
FROM 
    League_null AS l
FULL JOIN 
    Country AS c ON l.country_id = c.id;
'''
exe(c_sdb, full_join)


('Belgium Jupiler League', 'Belgium')
('England Premier League', None)
('France Ligue 1', None)
('Germany 1. Bundesliga', 'Germany')
('Italy Serie A', 'Italy')
('Netherlands Eredivisie', 'Netherlands')
('Poland Ekstraklasa', 'Poland')
('Portugal Liga ZON Sagres', 'Portugal')
('Scotland Premier League', 'Scotland')
('Spain LIGA BBVA', 'Spain')
('Switzerland Super League', 'Switzerland')
(None, 'Malta')
(None, 'England')
(None, 'France')


Observe the result and tell what you see.

Full join is another join that is not always implemented. It can ben performed (simulated) in different ways, by the combination (`UNION`) of different other selections. 

How could we obtain the same set ?

* We could select an inclusive left join + exclusive right join (that can be performed by a another left join btw)

or 

* we could select an inner join + exclusive left join + exclusive right join.

Exercice : try to implement a full join with those method (don’t forget that we can combine sets with `UNION ALL`).
Find [some hints here](https://www.sqlitetutorial.net/sqlite-full-outer-join/) if you get confused, but try by yourself first !



In [ ]:
# your code here




### Cross join

This is a special join. This join corresponds to the cartesian product between the rows of two tables, thus it needs no keys. It will produce a result where each line of the first table is related to each line of the second table.

```SQL
SELECT
  table1.column1,
  table2.column2
FROM
  table1
  CROSS JOIN table2;
```

Suppose you manage an association, and you want to produce an attendance sheet to note when members attended the reunions ?
1. Create a new data base called `club.sqlite`
2. Create a table `Members` with id, name (you can quickly create data by creating a `.csv` file, importing it and converting it to a table with `pandas`.)
3. Insert some members (3 or 4)
4. Create a table `Reunions` with id, reunion_date (you can set default to current datetime to save time)
5. Insert some reunions : you can save time by writing a script to fill the table with loop…
6. Write a query that produce a matrix with every reunion for each members (`CROSS JOIN`) ordered by date

In [23]:
# your code here







### Self join

Do you remember when in the first course, presenting relationships between tables, we presented the possibility that in one table, the primary key could be used as a foreign key to represent hierarchical relationship between entity inside the same table ? That’s what a self join do.

A self join is not specifically implemented. It can be simulated by joining the table on itself with an `INNER JOIN` or a `LEFT JOIN`. To do so, you have to use two *different* table alias for the same table to refer to it.

```SQL
SELECT e1.name as Managed, e2.name as Manager
FROM employee_table e1
INNER JOIN employee_table e2 ON e2.id = t1.Managed_by
```

Exercise : create a detailed table employee with such relation manager/managed. You can imagine other situation in which `SELF JOIN` would be pertinent. Create a request that shows for a list of employee their manager (take care of the appearance of the results, ordered by manager, you can use string function to present a phrase like : « X manages Y »). 

In [ ]:
# your code here




## Other advanced clauses

In this last section we present clauses that allow writing fairly complex queries while preserving a certain flexibility and readability. We present them quickly, because they will be especially useful to those who will be required to write advanced queries. If this is your case or if you want to master SQL in depth, do not hesitate to delve deeper into the subject by browsing the resources indicated at the beginning of this course, which remains above all introductory. And of course be sure to do the exercises on the subject (to follow in next sessions) with all the attention required ! 

### Conditions (2) : `CASE WHEN`

We have seen the `WHERE` keyword to implement conditional treatment in a query. But it can be used only to realize a conditional selection. If you want a behavior closer to the classical control structure `if then else` found in most of the programming languages, then `CASE` is for you. It evaluates an expressions is equal to a list of possible values, and if they are true or not, select different results / values to return :

```SQL
CASE <expression_to_evaluate>
     WHEN <condition_value_1> THEN <result_1>
     WHEN <condition_value_2> THEN <result_2>
     ...
     [ ELSE <else_result> ] 
END <result_variable_name>
```

In [20]:
case = '''
SELECT 
    id,
CASE 
    name
WHEN
    'England' 
    THEN
        'Outside CE'
WHEN
    'Scotland'
    THEN
        'Outside CE'
WHEN
    'Switzerland'
    THEN
        'EEE'
ELSE
    'CE'
END
    
FROM 
    Country;
'''

exe(c_sdb, case)

(1, 'CE')
(9, 'CE')
(1729, 'Outside CE')
(4769, 'CE')
(7809, 'CE')
(10257, 'CE')
(13274, 'CE')
(15722, 'CE')
(17642, 'CE')
(19694, 'Outside CE')
(21518, 'CE')
(24558, 'EEE')


Note : you can’t use Boolean operators in condition expression with this form of `CASE… WHEN` structure. 

There is a second form, even closer to `if then else`, where it tests boolean expressions :

In [36]:
case = '''
SELECT 
    id,
    player_name,
    height,
CASE 
WHEN
    height < 170
    THEN
        'short'
WHEN
    170 <= height AND height < 180
    THEN
        'average'
WHEN
    180 <= height AND height < 190
    THEN
        'tall'
ELSE
    'very tall'
END 
    AS height_category
    
FROM 
    Player
LIMIT 
    20;
'''

exe(c_sdb, case)

(1, 'Aaron Appindangoye', 182.88, 'tall')
(2, 'Aaron Cresswell', 170.18, 'average')
(3, 'Aaron Doran', 170.18, 'average')
(4, 'Aaron Galindo', 182.88, 'tall')
(5, 'Aaron Hughes', 182.88, 'tall')
(6, 'Aaron Hunt', 182.88, 'tall')
(7, 'Aaron Kuhl', 172.72, 'average')
(8, 'Aaron Lennon', 165.1, 'short')
(9, 'Aaron Lennox', 190.5, 'very tall')
(10, 'Aaron Meijers', 175.26, 'average')
(11, 'Aaron Mokoena', 182.88, 'tall')
(12, 'Aaron Mooy', 175.26, 'average')
(13, 'Aaron Muirhead', 187.96, 'tall')
(14, 'Aaron Niguez', 170.18, 'average')
(15, 'Aaron Ramsey', 177.8, 'average')
(16, 'Aaron Splaine', 172.72, 'average')
(17, 'Aaron Taylor-Sinclair', 182.88, 'tall')
(18, 'Aaron Wilbraham', 190.5, 'very tall')
(19, 'Aatif Chahechouhe', 175.26, 'average')
(20, 'Abasse Ba', 187.96, 'tall')


### Subqueries : `WITH`

#### Subqueries

Continuing the previous example, we now have the possibility to categorise players in four categories : `short`, `average`, `tall`, `very tall`. We may want to select and process data from these categories : in a certain manner, it’s like we would have created a new table and could run queries on it. It would be the same if we perform joins : it’s like we would have created new tables (or a view), and we would like to perform queries on them. In fact, that’s exactly how it works. We can chain – or more exactly nest – queries, and it’s how it has to be done to maintain readability and it’s a easier way to build complex queries.

Subqueries can be nested in `SELECT`, `INSERT`, `UPDATE` or `DELETE` statements (remember `CRUD`). We will focus in our example on the `SELECT` clause.

In this case, a subquery can be used :

* directly in the `SELECT` clause
* in the `FROM` clause
* in association with operators like : `IN`, `EXIST`, `ANY`, `ALL`, comparison… (RTFM if you want more precisions)

To nest queries, you just have to write them between parenthesis `( )`. You can define a variable name refering to the subquery by writing it directly after the closing parenthesis :


In [44]:
subquery = '''
SELECT 
    player_name,
    height,
    (
        SELECT 
            AVG(height)
        FROM 
            Player
    ) average_height
FROM 
    Player
WHERE 
    height > average_height 
ORDER BY 
    height DESC
LIMIT 
    10;
'''

exe(c_sdb, subquery)

('Kristof van Hout', 208.28, 181.86744484628662)
('Bogdan Milic', 203.2, 181.86744484628662)
('Costel Pantilimon', 203.2, 181.86744484628662)
('Fejsal Mulic', 203.2, 181.86744484628662)
('Jurgen Wevers', 203.2, 181.86744484628662)
('Kevin Vink', 203.2, 181.86744484628662)
('Lacina Traore', 203.2, 181.86744484628662)
('Nikola Zigic', 203.2, 181.86744484628662)
('Paolo Acerbis', 203.2, 181.86744484628662)
('Pietro Marino', 203.2, 181.86744484628662)


This example is useless of course (it’s an example), but it shows how it works, and how it adds flexibility. Subqueries are particularly useful when crossing data from multiple tables with joins, when constructing intermediate variables, etc.

#### `WITH`

The `WITH` clause just allows you to name a subquery in association with the keyword `AS`. Let‘s take the query that categorize player accoarding to their height :

In [47]:
with_subquery = '''
WITH 
    players_cat 
    AS (
        SELECT 
            id,
            player_name,
            height,
        CASE 
        WHEN
            height < 170
            THEN
                'short'
        WHEN
            170 <= height AND height < 180
            THEN
                'average'
        WHEN
            180 <= height AND height < 190
            THEN
                'tall'
        ELSE
            'very tall'
        END 
            AS height_category
            
        FROM 
            Player)

SELECT 
    player_name,
    height
FROM 
    players_cat
WHERE 
    height_category = 'tall' 
ORDER BY 
    height DESC
LIMIT 
    10;
'''

exe(c_sdb, with_subquery)

('Aaron Muirhead', 187.96)
('Abasse Ba', 187.96)
('Abdessalam Benjelloun', 187.96)
('Abdoulaye Diallo', 187.96)
('Abdoulaye Faye', 187.96)
('Abdoulaye Toure', 187.96)
('Abdullah Al Hafith', 187.96)
('Abella Perez Damia', 187.96)
('Adam Cummins', 187.96)
('Adam Duda', 187.96)


### Window functions 

According to the [SQLite documentation](https://sqlite.org/windowfunctions.html), "A window function is an SQL function where the input values are taken from a "window" of one or more rows in the results set of a SELECT statement." 

It is easier to understand window functions by comparing them to `GROUP BY`. Some functions like `AVG()` or `SUM()` make sense if we **aggregate** rows into groups over which we can now calculate average, sum, etc. But there are other functions for which we may wish to keep the individuality of the rows – because it makes sense to calculate the value for each row – but where the calculation for one row requires taking into account the values of the other rows. For example to calculate to which quantile (`NTILE()`) belongs a value in each line, you need not only to know the value for the current line, but also for the other lines :

![Window functions illustration](./images/WindowFunctions.png)

For example, let’s compute the quartiles (n=4) of the players height (that’s a better method than fix arbitrary limits) ? 

In [48]:
quartiles = '''

SELECT 
    player_name,
    height,
    NTILE(4) OVER (
        ORDER BY height
        ) quartiles
FROM 
    Player
LIMIT 
    20;
'''

exe(c_sdb, quartiles)

('Juan Quero', 157.48, 1)
('Diego Buonanotte', 160.02, 1)
('Maxi Moralez', 160.02, 1)
('Anthony Deroin', 162.56, 1)
('Bakari Kone', 162.56, 1)
('Edgar Salli', 162.56, 1)
('Fouad Rachid', 162.56, 1)
('Frederic Sammaritano', 162.56, 1)
('Lorenzo Insigne', 162.56, 1)
('Pablo Piatti', 162.56, 1)
('Quentin Othon', 162.56, 1)
('Samuel Asamoah', 162.56, 1)
('Aaron Lennon', 165.1, 1)
('Adrian Blad', 165.1, 1)
('Aidan Connolly', 165.1, 1)
('Albert Crusat', 165.1, 1)
('Aldo Kalulu', 165.1, 1)
('Alejandro Daro Gomez', 165.1, 1)
('Aleksander Jagiello', 165.1, 1)
('Ariel Ibagaza', 165.1, 1)


Here we only have the 20 shortest players (1st quartile) because as there is a lot of players, we have limite the results to 20 players and they are ordered by height. Another issue is that the number `1, 2, 3, 4` is not really readable. 

Let’s use a subquery to (pre-)select 40 players taken in the alphabetical order (we can assume that this will introduce some height variance) before calculating quartiles, and use a `CASE` structure to display `1st, 2d, 3rd, 4th` rather than raw integers :

In [55]:
quartiles = '''

SELECT 
    player_name,
    height,
    CASE NTILE(4) OVER (
        ORDER BY height
        )
        WHEN 1
            THEN '1st'
        WHEN 2
            THEN '2d'
        WHEN 3
            THEN '3rd'
        WHEN 4
            THEN '4th'
        END
            AS Quartile
FROM 
    (SELECT 
        *
     FROM 
         Player
     ORDER BY 
         player_name
    LIMIT 
        40);
'''

exe(c_sdb, quartiles)

('Aaron Lennon', 165.1, '1st')
('Aaron Cresswell', 170.18, '1st')
('Aaron Doran', 170.18, '1st')
('Aaron Niguez', 170.18, '1st')
('Aaron Kuhl', 172.72, '1st')
('Aaron Splaine', 172.72, '1st')
('Abdelmajid Oulmers', 172.72, '1st')
('Aaron Meijers', 175.26, '1st')
('Aaron Mooy', 175.26, '1st')
('Aatif Chahechouhe', 175.26, '1st')
('Aaron Ramsey', 177.8, '2d')
('Abdelaziz Barrada', 177.8, '2d')
('Abderrazak Jadid', 177.8, '2d')
('Abdoul Razzagui Camara', 177.8, '2d')
('Abdelhamid El Kaoutari', 180.34, '2d')
('Abdellah Zoubir', 180.34, '2d')
('Abdelmalek El Hasnaoui', 180.34, '2d')
('Abdou Traore', 180.34, '2d')
('Aaron Appindangoye', 182.88, '2d')
('Aaron Galindo', 182.88, '2d')
('Aaron Hughes', 182.88, '3rd')
('Aaron Hunt', 182.88, '3rd')
('Aaron Mokoena', 182.88, '3rd')
('Aaron Taylor-Sinclair', 182.88, '3rd')
('Abdelkader Ghezzal', 182.88, '3rd')
('Abdelouahed Chakhsi', 182.88, '3rd')
('Abdou Diallo', 182.88, '3rd')
('Abdoul Karim Yoda', 182.88, '3rd')
('Abdoulay Konko', 182.88, '3rd')

By writing such queries, which are divided into several sub-parts, you may value the possibility to use alias, define sub-queries, and the possibility of formatting the queries with many line breaks and indentations.

Other window functions are useful, especially to do descriptive statistics :

* `RANK()`
* `FIRST_VALUE()`
* `LAST_VALUE()`

You can also compute moving mean or running total through the window, using keywords like `BETWEEN`, `PRECEDING`, `FOLLOWING` to define the current window frame. By approaching such advanced uses, we have reached the limit of this introductory course, [here](https://www.sqlitetutorial.net/sqlite-window-functions/sqlite-window-frame/) is a very explicit tutorial if the subject of window frame interests you.

## Final tips

We will spend the last two sessions working on exercises to assimilate what you have seen.

1. Be analytical! Complex problems are made of simpler ones. Write your query step by step, define subqueries, aliases, etc.
2. Write pseudo-code first, use comments ! Once you have a clear idea of the steps, code them one after the other.
3. Test your code each time you add another step. If you test your code at the end, you may be overwhelmed with a lot of incomprehensible errors, that you could have resolved as you went along.
4. The objective of this course is to introduce you to SQL. Build solid fundations. Don’t try to master most complicated concepts or write the most sophisticated piece of code if you can’t write a proper simple query. It’s more valuable to master the 50 first percent of this course than misunderstanding 100 percent. You will learn SQL three times and forget it twice before mastering it. The most important is to understand the structure, the logic, the reasonning. When you will be faced with a SQL practical problem, you will know where and how to look for information on the right way to do it.
5. Code is like craft : you learn by doing. Code every day, get out of your comfort zone, work with data or on problems or project that motivate you.